In [ ]:
try:
  # This %tensorflow_version magic only works in Colab.
  %tensorflow_version 1.x
except Exception:
  pass
# For your non-Colab code, be sure you have tensorflow==1.15
import tensorflow as tf

tf.enable_eager_execution()

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = 96
BATCH_SIZE = 32

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.1,
        zoom_range=[0.9, 1.1],
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

val_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

train_flow = train_gen.flow_from_directory(
    '../data/input/Training',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    color_mode="grayscale")

val_flow = val_gen.flow_from_directory(
    '../data/input/PrivateTest',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    color_mode="grayscale")

In [ ]:
image_batch, label_batch = next(val_flow)
image_batch.shape, label_batch.shape
print("(min,max)=(", np.min(image_batch), ", ", np.max(image_batch), ")")

In [ ]:
print (train_flow.class_indices)

labels = '\n'.join(sorted(train_flow.class_indices.keys()))

with open('model/labels.txt', 'w') as f:
  f.write(labels)

In [ ]:
!cat labels.txt

In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 1)
input_tensor = tf.keras.layers.Input(shape=IMG_SHAPE)

# Create the base model from the pre-trained MobileNet V2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    input_tensor=input_tensor,
    include_top=False,
    weights=None)

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(filters=32, kernel_size=1, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=9, activation='softmax')
])

base_model.trainable = True
for layer in base_model.layers:
  layer.trainable =  True

for layer in model.layers:
    layer.trainable = True

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
base_model.summary()

In [ ]:
model.summary()

In [ ]:
history = model.fit_generator(
    train_flow, 
    epochs=60,
    steps_per_epoch=890, #28556/32
    validation_data=val_flow)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
from keras import backend as K

In [ ]:
saved_keras_model = 'model/builtin_mobilenetv2-longrun.h5'
model.save(saved_keras_model)

In [ ]:
REPDATA_PATH = '../data/input/Training/*/*'

# A generator that provides a representative dataset
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files(REPDATA_PATH)
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_png(image, channels=1)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    image = tf.cast(image, tf.float32)
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

def relu6(x):
  return K.relu(x, max_value=6)

# This code not work because of 
converter = tf.lite.TFLiteConverter.from_keras_model_file(saved_keras_model,
                                                         custom_objects={'relu6': relu6})
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

with open('model/builtin_mobilenetv2-longrun.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
batch_images, batch_labels = next(val_flow)

logits = model(batch_images)
prediction = np.argmax(logits, axis=1)
truth = np.argmax(batch_labels, axis=1)

keras_accuracy = tf.keras.metrics.Accuracy()
keras_accuracy(prediction, truth)

print("Raw model accuracy: {:.3%}".format(keras_accuracy.result()))

In [ ]:
def set_input_tensor(interpreter, input):
  input_details = interpreter.get_input_details()[0]
  tensor_index = input_details['index']
  scale, zero_point = input_details['quantization']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  # The input tensor data must be uint8: within [0, 255].
  input_tensor[:, :] = np.uint8(input / scale + zero_point)

def classify_image(interpreter, input):
  set_input_tensor(interpreter, input)
  interpreter.invoke()
  output_details = interpreter.get_output_details()[0]
  output = interpreter.get_tensor(output_details['index'])
  # Because the model is quantized (uint8 data), we dequantize the results
  scale, zero_point = output_details['quantization']
  output = scale * (output - zero_point)
  top_1 = np.argmax(output)
  return top_1

interpreter = tf.lite.Interpreter('model/builtin_mobilenetv2-longrun.tflite')
interpreter.allocate_tensors()

# Collect all inference predictions in a list
batch_prediction = []
batch_truth = np.argmax(batch_labels, axis=1)

for i in range(len(batch_images)):
  prediction = classify_image(interpreter, batch_images[i])
  batch_prediction.append(prediction)

# Compare all predictions to the ground truth
tflite_accuracy = tf.keras.metrics.Accuracy()
tflite_accuracy(batch_prediction, batch_truth)
print("Quant TF Lite accuracy: {:.3%}".format(tflite_accuracy.result()))